In [3]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

#import pytorch_lightning as pl
import warnings
warnings.filterwarnings("ignore")


df = pd.read_excel('./data/data.xlsx', skiprows=0)
del df['Unnamed: 0']

In [4]:
df.columns

Index(['date', 'value', 'electricity demand',
       'generation of energy from wind sources', 'is_weekend',
       'code of the day', 'value lag24', 'value lag48', 'value lag72',
       'value lag96', 'value lag120', 'value lag144', 'value lag168',
       'value lag336', 'generation of energy from wind sources lag24',
       'generation of energy from wind sources lag48',
       'generation of energy from wind sources lag72',
       'generation of energy from wind sources lag96',
       'generation of energy from wind sources lag120',
       'generation of energy from wind sources lag144',
       'generation of energy from wind sources lag168',
       'generation of energy from wind sources lag336',
       'electricity demand lag24', 'electricity demand lag48',
       'electricity demand lag72', 'electricity demand lag96',
       'electricity demand lag120', 'electricity demand lag144',
       'electricity demand lag168', 'electricity demand lag336'],
      dtype='object')

In [5]:
df = df.set_index("date", drop=True)

In [6]:
target = "value"
target_mean = df[target].mean()
target_stdev = df[target].std()

for c in df.columns:
    mean = df[c].mean()
    stdev = df[c].std()

    df[c] = (df[c] - mean) / stdev
    

# MLP

In [9]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split


y = df[[target]]
X = df.drop(columns=[target])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.0805, shuffle=False)

In [ ]:
model = MLPRegressor(random_state=1, activation='identity',  learning_rate='constant', learning_rate_init=0.0003,
                     early_stopping=True,  validation_fraction=0.0917, verbose=False, power_t=0.5, batch_size=32, alpha=0.0005, shuffle=False,
                      max_iter=60, solver='adam',  hidden_layer_sizes=(128,2)).fit(X_train, y_train)


In [ ]:
prediction = model.predict(X_test)
result = pd.DataFrame(y_test)
result["Model forecast"] = prediction
result = result.sort_index()


for c in result.columns:
    result[c] = result[c] * target_stdev + target_mean


df_result = result.reset_index(drop=False)

df_out = df_result[df_test_plot['date']>='2020-10-01 01:00:00']



In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
def calculate_metrics(df_test_plot):
    return {'mae' : mean_absolute_error(df_test_plot.value, df_test_plot['Model forecast']),
            'rmse' : mean_squared_error(df_test_plot.value, df_test_plot['Model forecast']) ** 0.5,
            'r2' : r2_score(df_test_plot.value, df_test_plot['Model forecast']),
             'mape' : mean_absolute_percentage_error(df_test_plot.value, df_test_plot['Model forecast'])*100}


result_metrics = calculate_metrics(df_test_plot)

# Prophet

In [ ]:
from prophet import Prophet

model = Prophet(weekly_seasonality=False, daily_seasonality=False, seasonality_mode='additive')

In [ ]:
ex_features = ['lag24', 'zap_lag24', 'zrodla_wiatrowe_lag24', "is_weekend", 'lag48', 'lag72', 'lag96', 'lag120', 'lag144','lag168', 'lag336', 'zap_lag48',
             'zrodla_wiatrowe_lag48', 'zap_lag72', 'zrodla_wiatrowe_lag72', 'zap_lag96', 'zrodla_wiatrowe_lag96',
              'zap_lag120', 'zrodla_wiatrowe_lag120', 'zap_lag144', 'zrodla_wiatrowe_lag144', 'zap_lag168', 'zrodla_wiatrowe_lag168',
              'zap_lag336', 'zrodla_wiatrowe_lag336', 'kod_dnia']

In [ ]:
for feature in ex_features:
    model.add_regressor(feature)
    
model.fit(df_train[["date", "value"] + ex_features].rename(columns={"date": "ds", "value": "y"}))

forecast = model.predict(df_test[["date", "value"] + ex_features].rename(columns={"date": "ds"}))

forecast_ = forecast.loc[:, forecast.columns.intersection(['ds','yhat'])]
forecast_ = forecast_.rename(columns={"ds": "date"})
df_test_ =  df_test.loc[:, df_test.columns.intersection(['date','value'])]
new_df = pd.merge(df_test_, forecast_, how='left', on=['date'])

new_df = new_df.set_index("date")

for c in new_df.columns:
    new_df[c] = new_df[c] * target_stdev + target_mean
    
df_test_plot = new_df

df_test_plot = df_test_plot.reset_index(drop=False)
df_test_plot = df_test_plot.rename(columns={"yhat": "Model forecast"})
